# Tasa de Servicio 


Cáulculo de la Tasa de Servicio para los proveedores CKD para su cumplimiento en la Expedición (AVIEXP), el Embarque (ETD) y la Llegada a puerto (ETA).

## Importar Librerias que se van a usar

In [204]:
#Importar Librerías
import pandas as pd
import numpy as np
import matplotlib as plt
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import os

## Cargar Archivos de Trabajo


* Archivos descargados desde SAP mediante la transacción VTT
* Archivo con los datos de la VTT de cada sourcing

1. Definir la ruta de los archivos

In [205]:
'''
#Define the path home
path=('C:/Users/Siberio/Desktop/Codigo TS/CSV/')
file_list = os.listdir(path)

#load the file with VTT info for each sourcing 
df_sourcings=pd.read_excel('C:/Users/Siberio/Desktop/Codigo TS/VTT FICHA.xlsx')
'''

"\n#Define the path home\npath=('C:/Users/Siberio/Desktop/Codigo TS/CSV/')\nfile_list = os.listdir(path)\n\n#load the file with VTT info for each sourcing \ndf_sourcings=pd.read_excel('C:/Users/Siberio/Desktop/Codigo TS/VTT FICHA.xlsx')\n"

In [206]:
#Define the path
path=('C:/Users/ax24491/Alliance/Aprovisionamiento Colombia - General/03. APPRO/02. CKD/Tasa de servicio/VTT Dashboard/Archivos VTT/CSV/')
file_list = os.listdir(path)

#load the file with VTT info for each sourcing 
df_sourcings=pd.read_excel('C:/Users/ax24491/Alliance/Aprovisionamiento Colombia - General/03. APPRO/02. CKD/Tasa de servicio/VTT Dashboard/VTT FICHA.xlsx')

#df_cambios=pd.read_excel('C:/Users/ax24491/Alliance/Aprovisionamiento Colombia - General/03. APPRO/02. CKD/Tasa de servicio/VTT Dashboard/Cambios VTT.xlsx')

#df_cambios

2. Combinar los archivos semanales

In [207]:
#Create a list for weekly files
dataframes = [] # Create a empty list
header_added = False  # To track if the header from the first file has been added

#Add all file names to a list
for file in file_list:
    if file.endswith('.csv'):
        file_path = os.path.join(path, file)
        df = pd.read_csv(file_path,sep=';',low_memory=False)
    
    #Add header of first file
    if not header_added:
        dataframes.append(df)  # Append the first DataFrame with header
        header_added = True
    else:
        dataframes.append(df.iloc[1:])  # Append subsequent DataFrames without header


# Concatenate all dataframes into a single dataframe
df_datos = pd.concat(dataframes, ignore_index=True)

3. Filtrar la información, dejando solo las columnas necesarias y eliminando las filas repetidas

In [208]:

#Delete unused columns
df_datos.drop(df_datos.columns[[0,1,3,6,7,8,9,10,11,12,13,14,15,16,17,18,20,21,22,23,24,25,26,27,30,31,33,34,35,36,37,38,39]],axis=1,inplace=True)
df_datos.drop(df_datos.columns[7],axis=1,inplace=True)

#Delete Suplementary RAN
df_datos = df_datos[df_datos['SemRAN'].str.contains('S') == False]

#Eliminar las Ran repetidas, conservando la que mas informacion contiene

# Define a function to select the row with the least number of null values
def select_row_least_nulls(group):
    null_counts = group.isnull().sum(axis=1)
    min_nulls = null_counts.min()
    return group[null_counts == min_nulls].iloc[0]

# Drop duplicates keeping the row with the least null values
df_datos = df_datos.groupby('RAN', group_keys=False).apply(select_row_least_nulls).reset_index(drop=True)

#Delete RAN´s without AVIEXP
df_datos = df_datos.dropna(subset=['F.Real Exp'])


4. Unir la información de SAP con la información de la VTT, incluyendo posibles cambios que se hagan durante el año

In [209]:
#Crear un df con las semanas a partir de las cuales aplican cambios a la VTT
df_cambios=df_sourcings[['Proveedor','Semana Cambio']].copy


df_cambios['Proveedor']=df_cambios['Proveedor'].astype(str)
df_sourcings['Proveedor']=df_sourcings['Proveedor'].astype(str)
df_sourcings['Semana Cambio']=df_sourcings['Semana Cambio'].astype(str)
df_sourcings['Concat']=df_sourcings['Proveedor'].str.cat(df_sourcings['Semana Cambio'])

df_datos['Proveedor']=df_datos['Proveedor'].astype(str)
df_final=pd.merge(df_datos,df_cambios,on='Proveedor',how='inner')
df_final['Semana Cambio']=df_final['Semana Cambio'].astype(int)
df_final['Numero S.Ran']=df_final['SemRAN'].str[1:3].astype(int)


def Calculate_VTT_to_apply(row):
    Cambio=0
    if int(row['Semana Cambio'])!=0:
        if int(row['Numero S.Ran']) < int(row['Semana Cambio']):
            Cambio=int(0)
        else:
            Cambio=row['Semana Cambio']
    return Cambio

df_final['Semana Cambio']=df_final.apply(Calculate_VTT_to_apply,axis=1)
df_final.drop(['Numero S.Ran'],axis=1,inplace=True)

df_final['Proveedor']=df_final['Proveedor'].astype(str)
df_final['Semana Cambio']=df_final['Semana Cambio'].astype(str)
df_final['Concat']=df_final['Proveedor'].str.cat(df_final['Semana Cambio'])
df_final

df_datos=pd.merge(df_final,df_sourcings,on='Concat',how='inner')
df_datos.drop(['Semana Cambio_x','Concat','Semana Cambio_y','Proveedor_y'],axis=1,inplace=True)
df_datos.rename(columns={'Proveedor_x':'Proveedor'},inplace=True)


C:\Users\ax24491\AppData\Local\Temp\ipykernel_12556\3118095415.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cambios['Proveedor']=df_cambios['Proveedor'].astype(str)


5. Reemplazar Valores para China (consolidación KWE)

In [211]:
#Asignar codigo de proveedor 9999 para los sourcings que se consolidan en KWE
dict_china={2063:9999,2165:9999,2212:9999,2953:9999,2999:9999,2201:9999,2164:9999,2157:9999,2266:9999,2273:9999}
dict_china

df_datos['Proveedor']=df_datos['Proveedor'].astype(int)
df_datos['Proveedor'].replace(dict_china,inplace=True)


Material              object
RAN                   object
SemRAN                object
Proveedor              int32
F.Real Exp            object
F.Real Emb            object
ETA                   object
Sourcing              object
RAN INICIAL AVIEXP     int64
RAN FINAL AVIEXP       int64
RAN ETD                int64
RAN INICIAL ETA        int64
RAN FINAL ETA          int64
DIA INICIAL EXP        int64
DIA FINAL EXP          int64
DIA ETD                int64
DIA INICIAL ETA        int64
DIA FINAL ETA          int64
dtype: object

## Cálculos

1. Calcular la última semana del año anterior al año en curso, para tener en cuenta años de 52 y 53 semanas

In [212]:
#Calculate the last week of year-1
def get_last_iso_week(year):
    year=int(year)
    last_day = datetime(year, 12, 31)
    weekday = last_day.weekday()
    days_to_subtract = (weekday - 3) % 7
    last_iso_week_start = last_day - timedelta(days=days_to_subtract)
    
    # Calcula el número de la semana ISO
    iso_week_number = last_iso_week_start.isocalendar()[1]
    return iso_week_number

current_year=datetime.now().year
past_year=current_year-1
last_iso_week_number = get_last_iso_week(past_year)

2. Calcular semanas según parametros VTT para las 3 tasas

In [213]:
#Calcular semana real AVIEXP
df_datos['F.Real Exp'] = pd.to_datetime(df_datos['F.Real Exp'], errors ='coerce',dayfirst=True)
df_datos['F.Real Exp'].astype('int64').dtypes

#Calcular semana real Embarque
df_datos['F.Real Emb'] = pd.to_datetime(df_datos['F.Real Emb'], errors ='coerce',dayfirst=True)
df_datos['F.Real Emb'].astype('int64').dtypes

#Calcular semana real Puerto
df_datos['ETA'] = pd.to_datetime(df_datos['ETA'], errors ='coerce',dayfirst=True)
df_datos['ETA'].astype('int64').dtypes

#Tomar solo el numero de la semana ran
df_datos['Numero S.Ran']=df_datos['SemRAN'].str[1:3].astype(float)

#Calculo Semana en la cual se va a considerar en el calculo de la Tasa
df_datos['Semana Tasa AVIEXP']= df_datos['Numero S.Ran']-df_datos['RAN FINAL AVIEXP']
df_datos['Semana Tasa ETD']=df_datos['Numero S.Ran']-df_datos['RAN ETD']
df_datos['Semana Tasa ETA']=df_datos['Numero S.Ran']-df_datos['RAN FINAL ETA']

3. Calcular días según parametros VTT para las 3 tasas

In [214]:
#Funcion para calcular una fecha ingresando el año, la semana y el día de la semana (1 lunes-7 domingo)
def find_date_by_iso_week(year, iso_week, weekday):
    if iso_week < 1 or iso_week > 53 or weekday < 1 or weekday > 7:
        raise ValueError('El número de semana ISO debe estar entre 1 y 53, y el número de día de la semana debe estar entre 1 y 7.')
    
    date = datetime(year, 1, 1)
    days_difference = weekday - date.isoweekday() % 7
    if days_difference < 0:
        days_difference += 7
    
    target_date = date + timedelta(days=days_difference + (iso_week - 1) * 7)
    return target_date

4. Calcular Status AVIEXP

In [215]:
def Calculate_Status_Aviexp(row):
    year=datetime.now().year
    sem_aviexp_estandar_i=row['Numero S.Ran']-row['RAN INICIAL AVIEXP']
    sem_aviexp_estandar_f=row['Numero S.Ran']-row['RAN FINAL AVIEXP']

#Calcular semana estandar en caso de que cambie de año
    
    if sem_aviexp_estandar_i<0:
        year=year-1
    if sem_aviexp_estandar_i <0:
        sem_aviexp_estandar_i=last_iso_week_number-abs(sem_aviexp_estandar_i)
    elif sem_aviexp_estandar_i ==0:
        sem_aviexp_estandar_i=last_iso_week_number

    if sem_aviexp_estandar_f <0:
        sem_aviexp_estandar_f=last_iso_week_number-abs(sem_aviexp_estandar_f)
    elif sem_aviexp_estandar_f ==0:
        sem_aviexp_estandar_f=last_iso_week_number


    target_date_i= find_date_by_iso_week(year, sem_aviexp_estandar_i, row['DIA INICIAL EXP']) #calcular fecha inicial AVIEXP segun VTT
    target_date_f= find_date_by_iso_week(year, sem_aviexp_estandar_f, row['DIA FINAL EXP']) #calcular fecha final AVIEXP segun VTT


    #Calcular status del Aviexp, en caso de que falten datos ejecuta la excepcion
    try:
        if row['F.Real Exp']>=target_date_i and row['F.Real Exp']<=target_date_f: 
            status_AVIEXP='A TIEMPO'  #Si la fecha de Aviexp está en el rango de la VTT
        elif row['F.Real Exp']>target_date_f: 
            status_AVIEXP='EN ATRASO'  #Si la fecha de Aviexp es posterior al rango de la VTT
        else:
            status_AVIEXP='EN ADELANTO'  #Si la fecha de Aviexp es anteior al rango de la VTT
    except:
         status_AVIEXP='SIN DATOS'  #Si no hay datos para calcular

    return status_AVIEXP

#Crear columna Status AVIEXP y aplicar en cada fila la funcion de calculo del status        
df_datos['Status AVIEXP']=df_datos.apply(Calculate_Status_Aviexp,axis=1)

5. Calcular Status Embarque

In [216]:
# Funcion para calcular Status de Embarque
def Calculate_Status_Embarque(row):
    year=datetime.now().year
    sem_embarque_estandar=row['Numero S.Ran']-row['RAN ETD']
    

#Calcular semana estandar en caso de que cambie de año
    if sem_embarque_estandar<0:
        year=year-1

    if sem_embarque_estandar <0:
        sem_embarque_estandar=last_iso_week_number-abs(sem_embarque_estandar)
    elif sem_embarque_estandar ==0:
        sem_embarque_estandar=last_iso_week_number

    


    target_date= find_date_by_iso_week(year, sem_embarque_estandar, row['DIA ETD']) #calcular fecha ETD segun VTT


    #Calcular status del Aviexp, en caso de que falten datos ejecuta la excepcion
    try:
        if row['F.Real Emb']==target_date: 
            status_Embarque='A TIEMPO'  #Si la fecha de Emabrque es igual a la VTT
        elif row['F.Real Emb']>target_date: 
            status_Embarque='EN ATRASO'  #Si la fecha de Emabrque es posterior a la la VTT
        else:
            status_Embarque='EN ADELANTO'  #Si la fecha de Emabrque es anteior a la VTT
    except:
         status_Embarque='SIN DATOS'  #Si no hay datos para calcular

    return status_Embarque


#Crear columna Status ETD y aplicar en cada fila la funcion de calculo del status        
df_datos['Status ETD']=df_datos.apply(Calculate_Status_Embarque,axis=1)


6. Calcular Status Llegada

In [217]:

# Funcion para calcular Status de Llegada a Puerto
def Calculate_Status_Llegada(row):
    year=datetime.now().year
    sem_ETA_estandar_i=row['Numero S.Ran']-row['RAN INICIAL ETA']
    sem_ETA_estandar_f=row['Numero S.Ran']-row['RAN FINAL ETA']

#Calcular semana estandar en caso de que cambie de año
    
    if sem_ETA_estandar_i<0:
        year=year-1

    if sem_ETA_estandar_i <0:
        sem_ETA_estandar_i=last_iso_week_number-abs(sem_ETA_estandar_i)
    elif sem_ETA_estandar_i ==0:
        sem_ETA_estandar_i=last_iso_week_number

    if sem_ETA_estandar_f <0:
        sem_ETA_estandar_f=last_iso_week_number-abs(sem_ETA_estandar_f)
    elif sem_ETA_estandar_f ==0:
        sem_ETA_estandar_f=last_iso_week_number


    target_date_i= find_date_by_iso_week(year, sem_ETA_estandar_i, row['DIA INICIAL ETA']) #calcular fecha inicial ETA segun VTT
    target_date_f= find_date_by_iso_week(year, sem_ETA_estandar_f, row['DIA FINAL ETA']) #calcular fecha final ETA segun VTT


    #Calcular status del ETA, en caso de que falten datos ejecuta la excepcion
    try:
        if row['ETA']>=target_date_i and row['ETA']<=target_date_f: 
            status_ETA='A TIEMPO'  #Si la fecha de ETA está en el rango de la VTT
        elif row['ETA']>target_date_f: 
            status_ETA='EN ATRASO'  #Si la fecha de ETA es posterior al rango de la VTT
        else:
            status_ETA='EN ADELANTO'  #Si la fecha de ETA es anteior al rango de la VTT
    except:
         status_ETA='SIN DATOS'  #Si no hay datos para calcular

    return status_ETA


df_datos['Status ETA']=df_datos.apply(Calculate_Status_Llegada,axis=1)

### Calculo Tasa de Servicio CKD

Aplicar filtros de sourcing y semanas

In [218]:
#Pasar df to numpy array

sourcings = df_sourcings.to_numpy()
final_file = df_datos.to_numpy()

#definir variables
sourcing_filter = []
week_filter = []
weeks = []
negative_weeks = []
status_filter = []
suppliers = df_datos["Proveedor"].unique()


# Filter by sourcings
def filter_sourcings():
    for supplier in suppliers: 
        supplier_aux = []
        for i in range(len(final_file)):
            if final_file[i,3] == supplier:
                supplier_aux.append(final_file[i])
        sourcing_filter.append(supplier_aux)


# Filter by sourcings and weeks. Columns 19, 20 and 21 are AVIEXP, ETD and ETA, respectively
def filter_weeks(week):
    for i in range(len(sourcing_filter)):
        supplier = np.array(sourcing_filter[i])
        min_week = np.min(supplier[:,week])
        max_week = np.max(supplier[:,week])
        if min_week == max_week:
            number_of_weeks = np.arange(0, int(max_week)+1)
        else: 
            number_of_weeks = np.arange(int(min_week), int(max_week)+1)
        weeks.append(number_of_weeks)
        weeks_by_sourcing = []
        for j in range(len(number_of_weeks)):
            weeks_aux = []
            for k in range(len(supplier)):
                if supplier[k,week] == number_of_weeks[j]:
                    weeks_aux.append(supplier[k])
            weeks_by_sourcing.append(weeks_aux)
        week_filter.append(weeks_by_sourcing)


# Filter by sourcings, weeks and status
# Status are defined by AVIEXP, ETD and ETA. This columns are 22, 23 and 24

def filter_by_status(week, status):
    for i in range(len(week_filter)):
        supplier = week_filter[i].copy()
        status_filter_aux = []
        for j in range(len(supplier)):
            filter = supplier[j]
            total = len(filter)
            if total != 0:
                in_advance = 0
                on_time = 0
                late = 0
                for k in range(total):
                    if filter[k][status] == "EN ADELANTO" and filter[k][week] > 0:
                        in_advance += 1
                    elif filter[k][status] == "A TIEMPO" and filter[k][week] > 0:
                        on_time += 1
                    elif filter[k][status] == "EN ATRASO" and filter[k][week] > 0:
                        late += 1
                ok = (in_advance + on_time)/total
                nok = late/total
            else:
                ok = float('nan')
            status_filter_aux.append(ok)
        status_filter.append(status_filter_aux)

Eliminar valores negativos para que no se considere en los cálculos

In [219]:
def negative_weeks_method(status_array):
    cont = 0
    negative_weeks = []

    for i in range(len(weeks)):
        cont = 0
        for j in range(len(weeks[i])):
            if weeks[i][j] <= 0:
                cont += 1
        negative_weeks.append(cont)
    
    for i in range(len(status_array)):
        if negative_weeks[i] != 0:
            for j in range(negative_weeks[i]):
                if negative_weeks[i] != 0:
                    status_array[i][j] = float('nan')
        start = weeks[i][0]
        end = weeks[i][-1]
        fee = status_array[i].copy()
        for j in range(int(start)):
            status_array[i][j] = float('nan')
        for k in range(len(fee)):
            if int(start) > len(status_array[i]):
                status_array[i].append(fee[k])
            else:
                status_array[i][int(start)-1] = fee[k]
            start += 1
    return status_array

In [220]:
def weigthed_averages(sourcings_array, week, status):
    weighted_averages_sourcing = []
    for sourcing in sourcings_array:
        if len(sourcing) != 0:
            in_advance = 0
            on_time = 0
            for i in range(len(sourcing)):
                if  sourcing[i][status] == "EN ADELANTO" and sourcing[i][week] > 0:
                        in_advance += 1
                elif sourcing[i][status] == "A TIEMPO" and sourcing[i][week] > 0:
                        on_time += 1
            ok = (in_advance + on_time)/len(sourcing)
            weighted_averages_sourcing.append(ok)
        else:
             weighted_averages_sourcing.append(float('nan'))
    return weighted_averages_sourcing

In [221]:
sourcing_filter = []
week_filter = []
weeks = []
negative_weeks = []
status_filter = []
filter_sourcings()
status_name = ["AVIEXP", "ETD", "ETA"]
weeks_number = [19, 20, 21]
status_number = [22, 23, 24]
suppliers_aux = suppliers.copy()
suppliers_aux = np.insert(suppliers_aux,0,0)


#Exortar archivos de las 3 tasas
for i in range(len(status_name)):
    week_filter = []
    weeks = []
    negative_weeks = []
    status_filter = []
    filter_weeks(weeks_number[i])
    filter_by_status(weeks_number[i],status_number[i])
    status_filter = negative_weeks_method(status_filter).copy()
    status_filter.insert(0,np.arange(1,53))
    weighted_average = weigthed_averages(sourcing_filter,weeks_number[i], status_number[i])
    weighted_average.insert(0,0)
    service_fee = pd.DataFrame(status_filter)
    service_fee.insert(0, 'Proveedores', suppliers_aux)
    service_fee.insert(1,'Promedio ponderado', weighted_average)
    service_fee = service_fee.transpose()
    #Exportar archivo
    service_fee.to_csv(status_name[i] +'.csv', index = False, header = False, sep=';')

1. Cambiar las semanas negativas por semanas del año anterior

In [222]:
'''
#Funcion para calcular la semana en caso de que cambie de año
def calculate_past_year(row,column,last_iso_week_number):
    year=datetime.now().year
    if int(row[column]) < 0:
        year=year-1

    if int(row[column]) <0:
        new_value=last_iso_week_number-abs(row[column])
    elif int(row[column]) ==0:
        new_value=last_iso_week_number
    else:
        new_value=row[column]

    return int(new_value)


# Convertir los valores negativos a semanas del año anterior
df_datos['Semana Tasa AVIEXP']=df_datos.apply(lambda row:calculate_past_year(row,'Semana Tasa AVIEXP',last_iso_week_number),axis=1)
df_datos['Semana Tasa ETD']=df_datos.apply(lambda row:calculate_past_year(row,'Semana Tasa ETD',last_iso_week_number),axis=1)
df_datos['Semana Tasa ETA']=df_datos.apply(lambda row:calculate_past_year(row,'Semana Tasa ETA',last_iso_week_number),axis=1)
'''

"\n#Funcion para calcular la semana en caso de que cambie de año\ndef calculate_past_year(row,column,last_iso_week_number):\n    year=datetime.now().year\n    if int(row[column]) < 0:\n        year=year-1\n\n    if int(row[column]) <0:\n        new_value=last_iso_week_number-abs(row[column])\n    elif int(row[column]) ==0:\n        new_value=last_iso_week_number\n    else:\n        new_value=row[column]\n\n    return int(new_value)\n\n\n# Convertir los valores negativos a semanas del año anterior\ndf_datos['Semana Tasa AVIEXP']=df_datos.apply(lambda row:calculate_past_year(row,'Semana Tasa AVIEXP',last_iso_week_number),axis=1)\ndf_datos['Semana Tasa ETD']=df_datos.apply(lambda row:calculate_past_year(row,'Semana Tasa ETD',last_iso_week_number),axis=1)\ndf_datos['Semana Tasa ETA']=df_datos.apply(lambda row:calculate_past_year(row,'Semana Tasa ETA',last_iso_week_number),axis=1)\n"

2. Exportar archivo consolidado cen formato csv

In [223]:
#Exportar como csv el dataframe con toda la informacion
#df_datos.to_csv('C:/Users/Siberio/Desktop/Codigo TS/Archivo_Final.csv',index=False,sep=';') #personal path
df_datos.to_csv(r'C:\Users\ax24491\Alliance\Aprovisionamiento Colombia - General\03. APPRO\02. CKD\Tasa de servicio\VTT Dashboard\Archivo_Final.csv',index=False,sep=';') #work path
print("finished")

finished
